In [160]:
import pandas as pd
from quantstats import extend_pandas
extend_pandas()

In [166]:
df = pd.read_csv("JALSH Index_dataset_2000_2024_clean.csv", index_col=0, header=[0, 1])
classfier = pd.read_excel("classification_data.xlsx", index_col=0)

In [334]:
classfier.head()

,name,sector,industry
ABG SJ Equity,Absa Group Ltd,Financial,Banks
ADH SJ Equity,Advtech Ltd,"Consumer, Non-cyclical",Commercial Services
AEL SJ Equity,Altron Ltd,Industrial,Electronics
AFE SJ Equity,AECI Ltd,Industrial,Miscellaneous Manufactur
AFH SJ Equity,Alexander Forbes Group Holding,Financial,Diversified Finan Serv


In [340]:
def filter_securities(filters, data=classfier):
    
    # Apply filters directly on the transposed
    filtered_data = data.query(
        ' and '.join(f'`{k}` == "{v}"' for k, v in filters.items())
    )
    
    return list(filtered_data.index)

filter_securities({'industry': 'Banks'})

['ABG SJ Equity',
 'FSR SJ Equity',
 'INP SJ Equity',
 'NED SJ Equity',
 'SBK SJ Equity']

In [196]:
profitability = ['ebitda_margin', 'oper_margin', 'return_on_asset']
liquidity = ['tot_debt_to_ebitda', 'tot_debt_to_tot_asset', 'tot_debt_to_tot_cap', 'tot_debt_to_tot_eqy', 'interest_coverage_ratio', 'ebitda_to_tot_int_exp']
efficiency = ['invent_to_sales', 'asset_turnover']

In [219]:
equally_weighted = {
    'profitability': {},
    'liquidity': {},
    'efficiency': {}
}
# Adjusted calculation to handle missing metrics
for group_name, metrics in [('profitability', profitability), ('liquidity', liquidity), ('efficiency', efficiency)]:
    for stock in df.columns.levels[0]:
        available_metrics = [metric for metric in metrics if metric in df[stock].columns]
        
        if available_metrics:  # Ensure there are available metrics to avoid division by zero
            equally_weighted[group_name][stock] = df[stock][available_metrics].mean(axis=1).mean()
        else:
            equally_weighted[group_name][stock] = None 

# Update the DataFrame with the adjusted calculations
equally_weighted_df_adjusted = pd.DataFrame(equally_weighted)
equally_weighted_df_adjusted.head()


,profitability,liquidity,efficiency
ABG SJ Equity,14.718020,109.257295,0.108163
ADH SJ Equity,13.873014,78.952831,0.756301
AEL SJ Equity,7.368305,22.197021,6.331297
AFE SJ Equity,8.135806,19.000728,9.441979
AFH SJ Equity,13.706324,30.938578,0.016101


In [239]:
equally_weighted_df_adjusted['profitability']

ABG SJ Equity    14.718020
ADH SJ Equity    13.873014
AEL SJ Equity     7.368305
AFE SJ Equity     8.135806
AFH SJ Equity    13.706324
                   ...    
TSG SJ Equity    22.870554
VKE SJ Equity    43.365386
VOD SJ Equity    26.181654
WBO SJ Equity     5.055584
WHL SJ Equity    10.006689
Name: profitability, Length: 124, dtype: float64

In [181]:
profitability = ['ebitda_margin', 'oper_margin', 'return_on_asset', ]
liquidity = ['tot_debt_to_ebitda', 'tot_debt_to_tot_asset', 'tot_debt_to_tot_cap', 'tot_debt_to_tot_eqy', 'interest_coverage_ratio', 'ebitda_to_tot_int_exp']
efficiency = ['invent_to_sales', 'asset_turnover']

In [212]:
# Initialize a dictionary to store the individual metric calculations
individual_metrics = {}

# List all metrics together for iteration
all_metrics = profitability + liquidity + efficiency

# Calculate the average of each metric individually for each stock
for stock in df.columns.levels[0]:
    individual_metrics[stock] = {}
    for metric in all_metrics:
        if metric in df[stock].columns:
            individual_metrics[stock][metric] = df[stock][metric].mean()

# Convert the results to a DataFrame for better presentation
individual_metrics_df = pd.DataFrame(individual_metrics).T

individual_metrics_df.head()


,oper_margin,return_on_asset,tot_debt_to_tot_asset,tot_debt_to_tot_cap,tot_debt_to_tot_eqy,asset_turnover,ebitda_margin,tot_debt_to_ebitda,interest_coverage_ratio,ebitda_to_tot_int_exp,invent_to_sales
ABG SJ Equity,28.202722,1.233318,20.212411,64.622361,242.937113,0.108163,NaN,NaN,NaN,NaN,NaN
ADH SJ Equity,14.421976,8.247937,19.850619,24.330877,38.061018,1.115640,18.949128,1.329016,170.789112,219.356343,0.396962
AEL SJ Equity,5.841907,7.653897,17.306101,30.528639,53.463897,1.468364,8.609112,1.722145,13.542255,18.225091,11.194231
AFE SJ Equity,8.061480,5.113338,21.207294,31.505856,47.459330,1.175394,11.232598,1.734404,5.032983,7.064499,17.708564
AFH SJ Equity,19.331788,0.099075,0.975794,31.281776,80.367497,0.016101,27.476468,2.281107,NaN,7.725572,NaN


In [310]:
individual_metrics_df.T.to_excel("individual_metrics.xlsx")

In [224]:
def compute_descriptive_stats(dataframe):
    """
    Computes descriptive statistics for each metric in the provided DataFrame.
    
    Parameters:
    - dataframe: pandas DataFrame with stocks as rows and metrics as columns.
    
    """
    # Calculate descriptive statistics with specified quantiles
    descriptive_stats = dataframe.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.95])
    
    # Add min and max values to ensure they are explicitly included
    min_values = dataframe.min()
    max_values = dataframe.max()
    
    # Combine min and max values with the descriptive statistics
    descriptive_stats.loc['min'] = min_values
    descriptive_stats.loc['max'] = max_values
    
    return descriptive_stats

# Example usage of the function
descriptive_stats_example = compute_descriptive_stats(individual_metrics_df)
descriptive_stats_example


,oper_margin,return_on_asset,tot_debt_to_tot_asset,tot_debt_to_tot_cap,tot_debt_to_tot_eqy,asset_turnover,ebitda_margin,tot_debt_to_ebitda,interest_coverage_ratio,ebitda_to_tot_int_exp,invent_to_sales
count,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,106.000000,106.000000,104.000000,106.000000,87.000000
mean,26.935606,8.030154,19.494478,30.485027,67.865970,0.840987,27.748282,4.453357,55.999754,69.060010,11.412251
std,44.586407,10.638520,12.612834,23.011801,77.627129,0.836852,23.777903,11.252781,166.694202,186.662963,8.570658
min,-15.613465,-1.982295,0.000000,0.000000,0.000000,0.016101,-13.418433,0.027125,-2.821364,-2.158024,0.000000
10%,4.351521,1.286708,3.166978,11.318676,14.113187,0.078841,6.572812,0.348460,1.586540,2.996644,0.478675
25%,7.751054,3.145640,9.651873,17.298927,24.450738,0.124893,11.662741,0.914727,3.602892,6.334164,3.758491
50%,16.638887,6.477793,18.417644,29.654190,44.974997,0.617119,20.192208,1.728275,7.772743,11.020836,10.881740
75%,28.805023,10.038253,28.038820,39.477491,73.431085,1.198899,35.302671,3.634308,19.821202,28.520319,16.653378
95%,73.352229,17.177836,39.014754,56.597518,230.230090,2.449184,76.276431,15.397066,208.373673,408.134542,24.394124
max,453.584997,109.384806,65.288960,218.185037,540.494050,4.312699,145.513193,98.162493,1339.192812,1496.857073,45.377134


In [339]:
individual_metrics_df[['tot_debt_to_tot_eqy']]

,ABG SJ Equity,ADH SJ Equity,AEL SJ Equity,AFE SJ Equity,AFH SJ Equity,AFT SJ Equity,AGL SJ Equity,AIL SJ Equity,AIP SJ Equity,AMS SJ Equity,...,TFG SJ Equity,TGA SJ Equity,THA SJ Equity,TKG SJ Equity,TRU SJ Equity,TSG SJ Equity,VKE SJ Equity,VOD SJ Equity,WBO SJ Equity,WHL SJ Equity
tot_debt_to_tot_eqy,242.937113,38.061018,53.463897,47.45933,80.367497,19.268366,43.240614,0.0,18.092223,20.964517,...,59.437803,2.451614,115.038914,56.593172,21.861382,141.812906,73.289447,65.00494,9.188145,124.097944


In [364]:
classfier.sector.value_counts()

sector
Financial                 45
Basic Materials           20
Consumer, Cyclical        20
Consumer, Non-cyclical    18
Industrial                 8
Communications             6
Technology                 3
Energy                     3
Diversified                1
Name: count, dtype: int64

In [368]:
individual_metrics_df.columns

Index(['oper_margin', 'return_on_asset', 'tot_debt_to_tot_asset',
       'tot_debt_to_tot_cap', 'tot_debt_to_tot_eqy', 'asset_turnover',
       'ebitda_margin', 'tot_debt_to_ebitda', 'interest_coverage_ratio',
       'ebitda_to_tot_int_exp', 'invent_to_sales'],
      dtype='object')

In [ ]:
classfier.sector.value_counts()

In [387]:
securities = filter_securities({"sector":"Communications", "industry": "Telecommunications"})
bloomberg_metrics = ['tot_debt_to_tot_eqy', 'interest_coverage_ratio', 'return_on_asset', 'tot_debt_to_ebitda', 'ebitda_to_tot_int_exp']
s = individual_metrics_df[bloomberg_metrics].T[securities].T

In [389]:
s.to_excel("data.xlsx")

In [391]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Load the dataset
data_path = '/path/to/your/data.xlsx'
df = s.copy()pd.read_excel(data_path)

# Calculate 25th and 75th quantiles for each metric
quantiles = df.quantile([0.25, 0.75])

# Function to classify based on quantiles
def classify(row, metric, quantiles):
    if row[metric] > quantiles.loc[0.75, metric]:
        return 2  # Good
    elif row[metric] < quantiles.loc[0.25, metric]:
        return 0  # Poor
    else:
        return 1  # Fair

# Apply the classification for a selected metric
metric_to_classify = 'tot_debt_to_tot_eqy'  # Example metric
df['classification'] = df.apply(classify, metric=metric_to_classify, quantiles=quantiles, axis=1)

# Define features and target
X = df[['interest_coverage_ratio', 'return_on_asset', 'tot_debt_to_ebitda', 'ebitda_to_tot_int_exp']]  # Example features
y = df['classification']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X_train_scaled, y_train)

# Predictions and classification report
predictions = model.predict(X_test_scaled)
print(classification_report(y_test, predictions))


,tot_debt_to_tot_eqy,interest_coverage_ratio,return_on_asset,tot_debt_to_ebitda,ebitda_to_tot_int_exp
BLU SJ Equity,32.809328,6.346575,4.215240,1.150158,7.547418
MTN SJ Equity,65.116785,8.547827,7.405181,1.305329,13.114130
TKG SJ Equity,56.593172,3.495262,6.401370,1.430977,8.796523
VOD SJ Equity,65.004940,9.790341,16.143841,0.793112,13.355185


In [392]:
df.apply(classify, metric=metric_to_classify, quantiles=quantiles, axis=1)

BLU SJ Equity    0
MTN SJ Equity    2
TKG SJ Equity    1
VOD SJ Equity    1
dtype: int64

In [393]:
quantiles

,tot_debt_to_tot_eqy,interest_coverage_ratio,return_on_asset,tot_debt_to_ebitda,ebitda_to_tot_int_exp
0.25,50.647211,5.633747,5.854837,1.060896,8.484247
0.75,65.032901,8.858455,9.589846,1.336741,13.174394


In [402]:
def classify(row, metric, quantiles, is_higher_good):
    val = row[metric]
    q25, q75 = quantiles.loc[0.25, metric], quantiles.loc[0.75, metric]
    good_condition = (val > q75) if is_higher_good else (val < q25)
    poor_condition = (val < q25) if is_higher_good else (val > q75)
    
    return 2 if good_condition else (0 if poor_condition else 1)

# Apply the classification for 'tot_debt_to_tot_eqy', considering lower values are better
df.apply(classify, metric='tot_debt_to_tot_eqy', quantiles=quantiles, is_higher_good=False, axis=1)

BLU SJ Equity    2
MTN SJ Equity    0
TKG SJ Equity    1
VOD SJ Equity    1
dtype: int64

In [399]:
df

,tot_debt_to_tot_eqy,interest_coverage_ratio,return_on_asset,tot_debt_to_ebitda,ebitda_to_tot_int_exp,classification
BLU SJ Equity,32.809328,6.346575,4.215240,1.150158,7.547418,0
MTN SJ Equity,65.116785,8.547827,7.405181,1.305329,13.114130,2
TKG SJ Equity,56.593172,3.495262,6.401370,1.430977,8.796523,1
VOD SJ Equity,65.004940,9.790341,16.143841,0.793112,13.355185,1


In [403]:
def classify_with_quantiles(df, metric, is_higher_good):
    """Classify the metric values into 'Good', 'Fair', or 'Poor' based on quantiles within the DataFrame.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing the metric to classify.
    metric (str): The metric name to classify.
    is_higher_good (bool): True if higher values of the metric are 'Good', False otherwise.
    
    Returns:
    pd.Series: A Series containing the classification for each row in the DataFrame.
    """
    quantiles = df[metric].quantile([0.25, 0.75])
    q25, q75 = quantiles[0.25], quantiles[0.75]
    
    def classify_value(val):
        good_condition = (val > q75) if is_higher_good else (val < q25)
        poor_condition = (val < q25) if is_higher_good else (val > q75)
        return 2 if good_condition else (0 if poor_condition else 1)
    
    return df[metric].apply(classify_value)

classify_with_quantiles(df, 'tot_debt_to_tot_eqy', False)


BLU SJ Equity    2
MTN SJ Equity    0
TKG SJ Equity    1
VOD SJ Equity    1
Name: tot_debt_to_tot_eqy, dtype: int64

In [412]:
df.columns

Index(['tot_debt_to_tot_eqy', 'interest_coverage_ratio', 'return_on_asset',
       'tot_debt_to_ebitda', 'ebitda_to_tot_int_exp', 'classification'],
      dtype='object')

In [419]:
financial_thresholds = {
  "tot_debt_to_tot_eqy": {
    1: (0, 0.5),
    0: (0.5, 2),
    -1: (2, float("inf")),
  },
  "interest_coverage_ratio": {
    1: (8, float("inf")),
    0: (3, 8),
    -1: (0, 3),
  },
  "return_on_asset": {
    1: (0.2, float("inf")),
    0: (0.05, 0.2),
    -1: (0, 0.05),
  },
  "tot_debt_to_ebitda": {
    1: (0, 2),
    0: (2, 4),
    -1: (4, float("inf")),
  },
  "ebitda_to_tot_int_exp": {
    1: (8, float("inf")),
    0: (3, 8),
    -1: (0, 3),
  },
}

In [420]:
def classify_stock_metrics(df, thresholds):
    classified_df = df.copy()

    for metric, threshold in thresholds.items():
        if metric in df.index:
            for category, bounds in threshold.items():
                lower, upper = bounds
                classified_df.loc[metric] = classified_df.loc[metric].apply(
                    lambda x: category if lower <= float(x) < upper else classified_df.loc[metric, classified_df.columns[0]]
                )
        else:
            print(f"Metric '{metric}' not found in the DataFrame. Skipping classification for this metric.")

    return classified_df

In [422]:
classify_stock_metrics(df.T, financial_thresholds).T

,tot_debt_to_tot_eqy,interest_coverage_ratio,return_on_asset,tot_debt_to_ebitda,ebitda_to_tot_int_exp,classification
BLU SJ Equity,-1.0,-1.0,1.0,1.0,-1.0,0.0
MTN SJ Equity,-1.0,0.0,1.0,1.0,0.0,2.0
TKG SJ Equity,-1.0,-1.0,1.0,1.0,0.0,1.0
VOD SJ Equity,-1.0,0.0,1.0,1.0,0.0,1.0


,tot_debt_to_tot_eqy,interest_coverage_ratio,return_on_asset,tot_debt_to_ebitda,ebitda_to_tot_int_exp
BLU SJ Equity,32.809328,6.346575,4.215240,1.150158,7.547418
MTN SJ Equity,65.116785,8.547827,7.405181,1.305329,13.114130
TKG SJ Equity,56.593172,3.495262,6.401370,1.430977,8.796523
VOD SJ Equity,65.004940,9.790341,16.143841,0.793112,13.355185


In [423]:
import pandas as pd
import numpy as np

def create_industry_thresholds(df, metric_directions):
    industry_thresholds = {}

    for metric in df.index:
        q25 = df.loc[metric].quantile(0.25)
        q75 = df.loc[metric].quantile(0.75)
        
        if metric_directions[metric] == "higher_better":
            industry_thresholds[metric] = {
                "Good": (q75, np.inf),
                "Fair": (q25, q75),
                "Weak": (-np.inf, q25)
            }
        elif metric_directions[metric] == "lower_better":
            industry_thresholds[metric] = {
                "Good": (-np.inf, q25),
                "Fair": (q25, q75),
                "Weak": (q75, np.inf)
            }
        else:
            raise ValueError(f"Invalid direction specified for metric '{metric}'")

    return industry_thresholds

def classify_stock_metrics(df, thresholds):
    classified_df = df.copy()

    for metric, threshold in thresholds.items():
        if metric in df.index:
            for category, bounds in threshold.items():
                lower, upper = bounds
                classified_df.loc[metric] = classified_df.loc[metric].apply(
                    lambda x: category if lower <= float(x) < upper else classified_df.loc[metric, classified_df.columns[0]]
                )
        else:
            print(f"Metric '{metric}' not found in the DataFrame. Skipping classification for this metric.")

    return classified_df

In [425]:
# Assuming you have a DataFrame called 'stock_metrics' with the metric values and stock symbols
metric_directions = {
    "tot_debt_to_tot_eqy": "lower_better",
    "interest_coverage_ratio": "higher_better",
    "return_on_asset": "higher_better",
    "tot_debt_to_ebitda": "lower_better",
    "ebitda_to_tot_int_exp": "higher_better"
}

industry_thresholds = create_industry_thresholds(stock_metrics, metric_directions)
print("Industry Thresholds:")
print(industry_thresholds)

classified_metrics = classify_stock_metrics(stock_metrics, industry_thresholds)
print("\nClassified Stock Metrics:")
print(classified_metrics)

{'tot_debt_to_tot_eqy': {'Good': (65.03290102599539, inf),
  'Fair': (50.647210600805735, 65.03290102599539),
  'Weak': (-inf, 50.647210600805735)},
 'interest_coverage_ratio': {'Good': (8.858455386626364, inf),
  'Fair': (5.633747012489427, 8.858455386626364),
  'Weak': (-inf, 5.633747012489427)},
 'return_on_asset': {'Good': (9.589846015144378, inf),
  'Fair': (5.854837291308153, 9.589846015144378),
  'Weak': (-inf, 5.854837291308153)},
 'tot_debt_to_ebitda': {'Good': (1.3367408348954506, inf),
  'Fair': (1.060896398933337, 1.3367408348954506),
  'Weak': (-inf, 1.060896398933337)},
 'ebitda_to_tot_int_exp': {'Good': (13.174393594464583, inf),
  'Fair': (8.484247144183694, 13.174393594464583),
  'Weak': (-inf, 8.484247144183694)},
 'classification': {'Good': (1.25, inf),
  'Fair': (0.75, 1.25),
  'Weak': (-inf, 0.75)}}